# Busca Cega

Algoritmos de Busca Cega, também conhecidos como Algoritmos de Busca Desinformados, não utilizam nenhum conhecimento prévio sobre o problema para auxiliar na busca. A principal desvantagem destes algoritmos é que eles não podem ser usados em problemas cuja natureza do espaço de estados seja exponencial. Mesmo no caso que o espaço de estados seja esparso, estes algoritmos executam em tempo hábil apenas para instâncias relativamente pequenas.

A implementação abaixo considera o modelo de busca proposto no livro do Russell e Norvig.

Estude os algoritmos com calma e depois faça os exercícios que estão propostos no final deste caderno.

In [ ]:
from pprint import pprint

## Estrutura Básica

In [ ]:
class Busca(object):
    """Esta classe é base para todas as buscas."""
    def __init__(self):
        pass
    
    def buscar(self, problema):
        """Buscar deve ser implementada para todas as buscas.
        
        Argumentos:
        
        problema -- Uma instância de uma subclasse de Problema.
        """
        raise NotImplementedError

    def solucao(self, no):
        """Esta função retorna a solução usada para chegar a *no*.
        Enquanto a busca acontece uma árvore de busca é construída (ver NoBusca).
        O caminho da solução é o caminho inverso da solução até a raíz da árvore.
        
        Argumentos:
        
        no -- O nó da arvore de busca que possui o estado objetivo.
        """
        p = no
        caminho = []
        while p:
            if p.acao:
                caminho.append(p.acao)
            else:
                caminho.append(p.estado)
            p = p.pai
        caminho.reverse()
        return (caminho, no.custo_total)

class NoBusca(object):
    """Esta classe é um container para a função CHILD-NODE, do livro do Russell.
    Ela representa um nó na árvore de busca que é gerada durante a busca.
    
    Atributos:
    
    estado -- O estado que este nó representa
    pai -- Aponta para o nó que gerou este atual.
    acao -- Aponta para a ação que gerou este nó.
    custo_total -- O custo total da solução desde a raíz até este nó.
    """
    def __init__(self, problema, estado, pai, acao, custo_total):
        self.estado = estado   
        self.pai = pai
        self.acao = acao
        self.custo_total = custo_total
    
    @classmethod
    def from_pai(cls, problema, pai, acao):
        estado = problema.resultado(pai.estado, acao)
        custo_total = pai.custo_total + problema.custo_passo(pai.estado, acao)
        return cls(problema, estado, pai, acao, custo_total)
    
    @classmethod
    def from_init(cls, problema):
        return cls(problema, problema.estado_inicial, None, None, 0)

class Estado(object):
    """Esta classe abstrata é uma casca pra representar estados.
    
    Atributos:
    
    chave -- Uma string que representa este estado. É usado para identificar o estado,
    portanto deve ser única para cada estado.
    """
    def __init__(self, chave):
        self.chave = chave

class Acao(object):
    """Esta classe abstrata é uma casca para representar ações.
    
    Atributos:
    
    chave -- Uma string que representa esta ação específica. É usada para identificar a ação,
    portanto deve ser única para cada ação possível a partir de cada estado.
    """
    def __init__(self, chave):
        self.chave = chave
        
class Problema(object):
    """Esta classe representa um problema a ser resolvido.
    
    Os 4 métodos marcados como não implementados devem ser implementados em uma subclasse.
    
    Atributos:
    
    estado_inicial -- Uma instância de classe derivada de Estado que representa por qual estado 
    a busca inicia.
    
    estado_final -- Uma instância de classe derivada de Estado que representa qual é o estado
    objetivo da busca.
    
    Métodos:
    
    custo_passo -- Retorna o custo de tomar a ação *acao* no estado *estado*.
    resultado -- Retorna o estado atingido a partir da ação *acao* no estado *estado*.
    Equivalente à função RESULT do livro do Russell.
    acoes -- Retorna uma lista de possíveis ações a serem tomadas a partir do estado *estado*.
    objetivo -- Retorna True se o estado *estado* é um estado final, caso contrário retorna False.
    """
    estado_inicial = None
    estado_final = None
    
    def __init__(self):
        pass
    
    def custo_passo(self, estado, acao):
        raise NotImplementedError
        
    def resultado(self, estado, acao):
        raise NotImplementedError
        
    def acoes(self, estado):
        raise NotImplementedError
    
    def objetivo(self, estado):
        raise NotImplementedError


## Descrição do Problema (Rotas da Romênia)

In [ ]:
class EstadoEmRota(Estado):
    """Representa um estado no espaço de busca do problema de Rotas da Romênia.
    
    No caso deste problema, somente uma informação está atribuída ao estado, o nome da cidade. Neste caso,
    usamos o próprio nome da cidade como chave, uma vez que não há duas cidades diferentes com o mesmo nome.
    """
    def __init__(self, chave):
        super(EstadoEmRota, self).__init__(chave)
        
    def __repr__(self):
        return ("Em(%s)" % str.capitalize(self.chave))
    
    def __str__(self):
        return self.__repr__()

class AcaoIrRota(Acao):
    """Representa ações possíveis a partir de dada cidade.
    
    No caso do problema do mapa da Romênia, temos 2 informações relacionadas com cada ação:
    o estado destino e o custo (distancia em km até o destino) da ação.
    """
    
    def __init__(self, chave, destino, custo):
        super(AcaoIrRota, self).__init__(chave)
        self.destino = destino
        self.custo = custo
        
    def __repr__(self):
        return ("Ir(%s,%d)" % (str.capitalize(self.chave), self.custo))
    
    def __str__(self):
        return self.__repr__()
    
class RotasRomenia(Problema):
    """Esta classe representa o problema das rotas entre as cidades da Romênia.
    
    Basta implementar os métodos *objetivo*, *acoes*, *resultado* e *custo_passo* para que as buscas funcionem.
    Este modelo é baseado na abordagem do livro do Russell. Ver o livro para detalhes.
    """
    
    def __init__(self):
        pass
        
    @classmethod
    def from_arquivo(cls, arquivo, chave_inicial=None,chave_final=None):
        """Este método lê a função de transição de estados a partir de um arquivo.
        
        A função de transição é representada como segue: um dicionário *estados* é indexado pela chave de todos
        os estados do problema. O valor de cada chave é uma lista das possíveis ações que podem ser executadas
        a partir do estado. Com esta estrutura, a implementação dos métodos *objetivo*, *acoes*, *resultado* e *custo_passo*
        é trivial.
        """
        
        #Abrir o arquivo com as transições de estado.
        with open(arquivo,'r') as f:
            estradas = f.readlines()
        estradas = list(map(str.strip,estradas))
        estradas = list(map(str.split,estradas,[','] * len(estradas)))
        
        #As duas primeiras linhas tem a chave dos estados inicial e final
        cls.estado_inicial = EstadoEmRota(estradas[0][0]) if not chave_inicial else EstadoEmRota(chave_inicial)
        cls.estado_final = EstadoEmRota(estradas[1][0]) if not chave_final else EstadoEmRota(chave_final)
        
        #criar o dicionário de estados. note que cada chave corresponde a uma tupla (estado,lista)
        #onde estado é uma instância derivada de Estado que corresponde à chave e [] é uma lista de ações, 
        #derivadas de Acao.
        estados = {cls.estado_inicial.chave : (cls.estado_inicial, []), cls.estado_final.chave : (cls.estado_final, [])}
        
        #para cada estrada,
        for caminho in estradas[2:]:
            #extrair o estado de origem
            if caminho[0] in estados:
                origem = estados[caminho[0]]
            else:
                estados[caminho[0]] = (EstadoEmRota(caminho[0]), [])
                origem = estados[caminho[0]]
                
            #extrair o estado de destino
            if caminho[1] in estados:
                destino = estados[caminho[1]][0]
            else:
                estados[caminho[1]] = (EstadoEmRota(caminho[1]), [])
                destino = estados[caminho[1]][0]
            
            #extrair o custo da transição
            custo = int(caminho[2])
            
            #Inserir a rota do até o destino na lista de ações do estado origem
            acao = AcaoIrRota(destino.chave, destino, custo)
            origem[1].append(acao)
                
        cls.estados = estados         
            
        return cls()
        
    def objetivo(self, estado):
        """Verifica se *estado* é um estado final da busca. 
        
        Argumentos:
        
        *estado* é uma instância de EstadoEmRota.
        """
        return True if estado.chave == self.estado_final.chave else False
        
    def acoes(self, estado):
        """Retorna todas as ações possíveis a partir do estado atual *estado*. As ações são instâncias de
        AcaoIrRota.
        
        Argumentos:
        
        *estado* -- uma instância de EstadoEmRota
        """
        return self.estados[estado.chave][1]
        
    def resultado(self, estado, acao):
        """Retorna o estado que é atingido a partir do estado atual *estado* e ação *acao*.
        
        Argumentos:
        
        *estado* -- uma instância de EstadoEmRota
        *acao* -- uma instância de AcaoIrRota
        """
        return acao.destino
    
    def custo_passo(self, estado, acao):
        """Retorna o custo de tomar a ação *acao* a partir do estado *estado*.
        
        Argumentos:
        
        *estado* -- uma instância de EstadoEmRota
        *acao* -- uma instância de AcaoIrRota
        """
        return acao.custo
        

## Funções Auxiliares

In [ ]:
def prints_no(n):
    if n.pai:
        return ("Estado: %s, Pai: %s, Ação: %s, Custo Acumulado: %d" % (n.estado, n.pai.estado, n.acao, n.custo_total))
    else:
        return ("Estado: %s, (estado inicial), Custo: 0" % (n.estado))

def print_nos(ns):
    for n in ns:
        print('\t' + prints_no(n))
    
def print_fronteira(fronteira):
    print("Fronteira:")
    print_nos(fronteira)

def print_explorados(explorados):
    print("Explorados:")
    print('\t' + str([x for x in explorados]))

def print_acoes(acoes):
    print("Ações possíveis:")
    print('\t' + str(acoes))

## Busca em Largura

In [ ]:
class BuscaLargura(Busca):
    """Esta classe implementa a busca em largura. (note a semelhança com o pseudocódigo apresentado na aula)
    
    Note que ela utiliza os 4 métodos *objetivo*, *acoes*, *resultado* e *custo_passo* de uma classe derivada de
    Problema. Se respeitar as restrições descritas em Problema, esta busca pode ser usada em qualquer problema.
    """
    
    def buscar(self, problema):
        #cria um nó da árvore de busca com o estado inicial
        no = NoBusca.from_init(problema)
        #verifica se já está no objetivo
        if problema.objetivo(no.estado):
            #caso esteja, retorna a solução
            return self.solucao(no)
        #inicializa a fronteira com o nó inicial. A fronteira é uma estrutura FIFO.
        fronteira = [no]
        #inicializa o dicionário (hash) de estados explorados como vazio
        explorado = dict()
        it = 0
        while True:
            print('####Iteração %d####' % (it))
            #Se a fronteira estiver esgotada, então a busca falhou. Só deve acontecer se o estado não existe
            #no espaço de estados.
            print_fronteira(fronteira)
            if len(fronteira) == 0:
                return False
            #Pega o primeiro nó da fronteira.
            no = fronteira.pop(0)
            print('NÓ ESCOLHIDO: %s' % (prints_no(no)))
            #Marcar o estado que corresponde ao nó atual como já visitado.
            explorado[no.estado.chave] = no.estado
            print_explorados(explorado)
            print_acoes(problema.acoes(no.estado))
            #Para todas as ações possíveis a partir do estado atual
            expandidos = []
            for acao in problema.acoes(no.estado):
                #criar um nó da árvore de busca (e conectá-lo) com o estado atingido a partir da ação
                filho = NoBusca.from_pai(problema, no, acao)
                #Somente considerar o filho pra ser explorado se não tiver sido explorado anteriormente
                #E se já não estiver na fronteira! (essa linha feia com len,list,filter e lambda faz a segunda
                #verificação)
                if (filho.estado.chave not in explorado) and \
                    (not len(list(filter(lambda x: x.estado.chave == filho.estado.chave, fronteira))) > 1):
                    expandidos.append(filho)
                    #Se o estado do nó filho gerado por um estado final,
                    if problema.objetivo(filho.estado):
                        #retorna a solução!
                        return self.solucao(filho)
                    #Caso contrário, adicionar o nó filho na fronteira.
                    fronteira.append(filho)
            print("Nós expandidos (%d):" % (len(expandidos)))
            if len(expandidos) > 0:
                print_nos(expandidos)
            else:
                print("\tNenhum!")
            it+=1
            print("")
            

## Busca em Profundidade

In [ ]:
class BuscaProfundidade(Busca):
    """Esta classe implementa a busca em profundidade. (procure a diferença com a busca em largura!)
    
    Note que ela utiliza os 4 métodos *objetivo*, *acoes*, *resultado* e *custo_passo* de uma classe derivada de
    Problema. Se respeitar as restrições descritas em Problema, esta busca pode ser usada em qualquer problema.
    """
    
    def buscar(self, problema):
        #cria um nó da árvore de busca com o estado inicial
        no = NoBusca.from_init(problema)
        #verifica se já está no objetivo
        if problema.objetivo(no.estado):
            #caso esteja, retorna a solução
            return self.solucao(no)
        #inicializa a fronteira com o nó inicial. A fronteira é uma estrutura FIFO.
        fronteira = [no]
        #inicializa o dicionário (hash) de estados explorados como vazio
        explorado = dict()
        it = 0
        while True:
            print('####Iteração %d####' % (it))
            #Se a fronteira estiver esgotada, então a busca falhou. Só deve acontecer se o estado não existe
            #no espaço de estados.
            print_fronteira(fronteira)
            if len(fronteira) == 0:
                return False
            #Pega o último nó inserido na fronteira
            no = fronteira.pop()
            print('NÓ ESCOLHIDO: %s' % (prints_no(no)))
            #Marcar o estado que corresponde ao nó atual como já visitado.
            explorado[no.estado.chave] = no.estado
            print_explorados(explorado)
            print_acoes(problema.acoes(no.estado))
            #Para todas as ações possíveis a partir do estado atual
            expandidos = []
            for acao in problema.acoes(no.estado):
                #criar um nó da árvore de busca (e conectá-lo) com o estado atingido a partir da ação
                filho = NoBusca.from_pai(problema, no, acao)
                #Somente considerar o filho pra ser explorado se não tiver sido explorado anteriormente
                #E se já não estiver na fronteira! (essa linha feia com len,list,filter e lambda faz a segunda
                #verificação)
                if (filho.estado.chave not in explorado) and \
                    (not len(list(filter(lambda x: x.estado.chave == filho.estado.chave, fronteira))) > 1):
                    expandidos.append(filho)
                    #Se o estado do nó filho gerado por um estado final,
                    if problema.objetivo(filho.estado):
                        #retorna a solução!
                        return self.solucao(filho)
                    #Caso contrário, adicionar o nó filho na fronteira.
                    fronteira.append(filho)
            print("Nós expandidos (%d):" % (len(expandidos)))
            if len(expandidos) > 0:
                print_nos(expandidos)
            else:
                print("\tNenhum!")
            it+=1
            print("")

## Programa Principal

In [ ]:
p = RotasRomenia.from_arquivo('romenia.txt', chave_inicial='neamt', chave_final='timisoara')  

bl = BuscaLargura()

print("Busca em Largura:")
solucao, custo = bl.buscar(p)

print("\nSolução:")
print("\tCaminho: %s" % str(solucao))
print("\tCusto total: %d" % custo)

print("")

print("Busca em Profundidade:")
bp = BuscaProfundidade()
solucao, custo = bp.buscar(p)

print("\nSolução:")
print("\tCaminho: %s" % str(solucao))
print("\tCusto total: %d" % custo)

# Exercícios

**a)** Acompanhe a execução da *Busca em Largura* e da *Busca em Profundidade* para os pares de (partida x destino) a seguir:

1. Arad x Bucharest
2. Craiova x Neamt
3. Oradea x Eforie
4. Oradea x Iasi
5. Sibiu x Timisoara

Para cada um, indique para ambos algoritmos de busca:
 
* A quantidade de iterações para finalizar a busca;
* Quantidade de cidades na solução;
* O custo total da solução;
* O tamanho médio da fronteira durante a busca;
* Se você fizer a busca A x B e B x A, a rota encontrada é a mesma? Por quê?

**b)** Na sala de aula vimos mais dois algoritmos de busca cega: *Busca de Custo Uniforme* e *Busca Por Aprofundamento Iterativo*. 

1. Escolha um deles e implemente usando a estrutura que já está montada neste caderno.
2. Refaça os testes do exercício **a** usando sua implementação.

**Dicas:** para implementar uma nova busca você não precisa **(e não deve)** alterar nenhuma das outras classes. Veja como foram implementadas Busca em Largura e Busca em Profundidade: compare com as versões apresentadas nas aulas. Basta herdar a classe *Busca* e reimplementar o método *buscar*.

**c)** Na sala de aula fizemos o projeto dos problemas dos *Canibais e Missionários* e da *Torre de Hanói*.

1. Implemente a descrição de um desses problemas conforme a estrutura usada no problema das Rotas da Romênia.
2. Teste os 3 algoritmos deste caderno (Busca em Largura, Busca em Profundidade e o algoritmo de busca que você implementou no exercício **b**). Responda os mesmos itens do exercício **a**.

**Dicas:** para implementar a descrição de um problema você não precisa **(e nem deve)** alterar nenhuma das outras classes. Veja como foi descrito o problema das rotas da Romênia. Você vai precisar herdar as classes Acao, Estado e Problema (ver a Seção Descrição do Problema).

# Boa Diversão!